# Model Discrimination for  R32/emimTF2N system

EoS: PR vs. SRK

Parameter T dependence: Linear

N (total fitting parameters): 4

## Import Functions


In [1]:
import idaes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize

import sys
sys.path.append('../')

from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)
from pyomo.opt import TerminationCondition, SolverStatus

from idaes.core import FlowsheetBlock
import idaes.logger as idaeslog
# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)
# Import unit models from the model library
from idaes.generic_models.unit_models import Flash
# Import degrees of freedom tool
from idaes.core.util.model_statistics import degrees_of_freedom

import scipy.stats as stats

## Make model functions

In [2]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [3]:
def calc_PRmod_P(x_HFC,T):
    '''
    Define function to calculate the PR 4 parameter model pressure given a composition and temperature
    
    Inputs:
    x_HFC - HFC compositions
    T - temperature to run
    
    Output: 
    P: pressure
    
    '''
    
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})
    
    # Configuration
    from hfc32_emimtf2n_PR import configuration as config_PR

    m.fs.properties = GenericParameterBlock(default=config_PR)

    m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                               "has_heat_transfer": True,
                               "has_pressure_change": True})

    m.fs.F101.inlet.flow_mol.fix(1)
    m.fs.F101.inlet.temperature.fix(323.15)
    m.fs.F101.inlet.pressure.fix(399800)
    m.fs.F101.inlet.mole_frac_comp[0,'R32'].fix(0.5)
    
    # Model params
    params_PR = pd.read_csv('../Params/PR_params_linTdep.csv',header=None)
    
    m.fs.properties.PR_kappa_A['R32','emimTf2N'] = float(params_PR.iloc[0])
    m.fs.properties.PR_kappa_A['emimTf2N','R32'] = float(params_PR.iloc[1])
    m.fs.properties.PR_kappa_B['R32','emimTf2N'] = float(params_PR.iloc[2])
    m.fs.properties.PR_kappa_B['emimTf2N','R32'] = float(params_PR.iloc[3])
    
    # Initialize the flash unit
    m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)

    # Fix the state variables on the state block
    # m.fs.F101.inlet.pressure.unfix()
    m.fs.F101.inlet.mole_frac_comp[0,'emimTf2N'].unfix()

    m.fs.liq = Param(mutable=True,default=0.040)
    m.fs.liquid = Constraint(expr=m.fs.F101.liq_outlet.mole_frac_comp[0, "emimTf2N"] == m.fs.liq)
    
    # Calc pressure
    x_IL = 1-x_HFC
    if 0.05 <= x_HFC <= 0.09:
        guess_P = 90000
    elif 0.09 < x_HFC <= 0.15:
        guess_P = 100100
    elif 0.15 < x_HFC <= 3:
        guess_P = 250000
    else:
        guess_P = 400000
    
    m.fs.liq = x_IL
    m.fs.F101.inlet.temperature.fix(T)
    m.fs.F101.inlet.pressure.fix(guess_P)
    m.fs.F101.inlet.mole_frac_comp[0,'R32'].fix(x_HFC+0.1)
    m.fs.F101.inlet.mole_frac_comp[0,'emimTf2N'].fix(float(1-(x_HFC+0.1)))
    m.fs.F101.vap_outlet.temperature.fix(float(T))
    
    try:
        m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)
        
    except:
        print('Initialization Error')
        
    try:

        status = solver.solve(m, tee = False)
        
        if (status.solver.status == SolverStatus.ok) and (status.solver.termination_condition == TerminationCondition.optimal):
            print('Feasible/Optimal Solution')
            x_HFC_final = value(m.fs.F101.liq_outlet.mole_frac_comp[0,'R32']) 
            P_final = value(m.fs.F101.vap_outlet.pressure[0])
        else:
            print('Infeasible Solution')
            x_HFC_final = np.nan
            P_final = np.nan
            
    except ValueError:
        x_HFC_final = np.nan
        P_final = np.nan
        print('Solver Error')
    
    return P_final

In [4]:
def calc_SRKmod_P(x_HFC,T):
    '''
    Define function to calculate the SRK 4 parameter model pressure given a composition and temperature
    
    Inputs:
    x_HFC - HFC compositions
    T - temperature to run
    
    Output: 
    P: pressure
    
    '''
    
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})
    
    # Configuration
    from hfc32_emimtf2n_SRK import configuration as config_SRK

    m.fs.properties = GenericParameterBlock(default=config_SRK)

    m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                               "has_heat_transfer": True,
                               "has_pressure_change": True})

    m.fs.F101.inlet.flow_mol.fix(1)
    m.fs.F101.inlet.temperature.fix(323.15)
    m.fs.F101.inlet.pressure.fix(399800)
    m.fs.F101.inlet.mole_frac_comp[0,'R32'].fix(0.5)
    
    # Model params
    params_SRK = pd.read_csv('../Params/SRK_params_linTdep.csv',header=None)
    
    m.fs.properties.SRK_kappa_A['R32','emimTf2N'] = float(params_SRK.iloc[0])
    m.fs.properties.SRK_kappa_A['emimTf2N','R32'] = float(params_SRK.iloc[1])
    m.fs.properties.SRK_kappa_B['R32','emimTf2N'] = float(params_SRK.iloc[2])
    m.fs.properties.SRK_kappa_B['emimTf2N','R32'] = float(params_SRK.iloc[3])
    
    # Initialize the flash unit
    m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)

    # Fix the state variables on the state block
    # m.fs.F101.inlet.pressure.unfix()
    m.fs.F101.inlet.mole_frac_comp[0,'emimTf2N'].unfix()

    m.fs.liq = Param(mutable=True,default=0.040)
    m.fs.liquid = Constraint(expr=m.fs.F101.liq_outlet.mole_frac_comp[0, "emimTf2N"] == m.fs.liq)
    
    # Calc pressure
    x_IL = 1-x_HFC
    if 0.05 <= x_HFC <= 0.09:
        guess_P = 90000
    elif 0.09 < x_HFC <= 0.15:
        guess_P = 100100
    elif 0.15 < x_HFC <= 3:
        guess_P = 250000
    else:
        guess_P = 400000
    
    
    m.fs.liq = x_IL
    m.fs.F101.inlet.temperature.fix(T)
    m.fs.F101.inlet.pressure.fix(guess_P)
    m.fs.F101.inlet.mole_frac_comp[0,'R32'].fix(x_HFC+0.1)
    m.fs.F101.inlet.mole_frac_comp[0,'emimTf2N'].fix(float(1-(x_HFC+0.1)))
    m.fs.F101.vap_outlet.temperature.fix(float(T))
    
    try:
        m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)
        
    except:
        print('Initialization Error')
        
    try:

        status = solver.solve(m, tee = False)
        
        if (status.solver.status == SolverStatus.ok) and (status.solver.termination_condition == TerminationCondition.optimal):
            print('Feasible/Optimal Solution')
            x_HFC_final = value(m.fs.F101.liq_outlet.mole_frac_comp[0,'R32']) 
            P_final = value(m.fs.F101.vap_outlet.pressure[0])
        else:
            print('Infeasible Solution')
            x_HFC_final = np.nan
            P_final = np.nan
            
    except ValueError:
        x_HFC_final = np.nan
        P_final = np.nan
        print('Solver Error')
    
    return P_final

In [5]:
def error(params):
    '''
    Function which calculates the squared error between two models
 
    Inputs:
    params:
        x_HFC - HFC compositions
        T - temperature to run
    
    Ouputs:
    error - squared error between two models, made negative
    '''
    
    x_HFC = params[0]
    T = params[1]
    
    print('Trying x_HFC:',x_HFC)
    print('Trying T:',T)
    
    P_PR = calc_PRmod_P(x_HFC,T)
    
    P_SRK = calc_SRKmod_P(x_HFC,T)
    
    error = (P_PR - P_SRK)**2
    
    print('error:',error)
    
    return error*(-1)

## Setup Optimization Problem for Model Discrimination

In [6]:
var_guess = (0.1,280)
bnds = ((0.05,0.8),(275,360))
results = scipy.optimize.minimize(error,var_guess,bounds=bnds,tol=1E-4)

Trying x_HFC: 0.1
Trying T: 280.0
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 613.128450262816
Trying x_HFC: 0.10000001
Trying T: 280.0
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 613.1286900577907
Trying x_HFC: 0.1
Trying T: 280.00000001
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 613.1284473110235
Trying x_HFC: 0.8
Trying T: 275.0
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 619066.005915399
Trying x_HFC: 0.79999999
Trying T: 275.0
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 619066.1591615747
Trying x_HFC: 0.8
Trying T: 275.00000001
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 619066.0036672559
Trying x_HFC: 0.10132860598182089
Trying T: 279.9905099572727
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 647.9954195176696
Trying x_HFC: 0.10132861598182089
Trying T: 279.9905099572727
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 647.9956704899299
Trying x_HFC: 0.10132860598182

Feasible/Optimal Solution
error: 1411396.5780016144
Trying x_HFC: 0.6958293468536605
Trying T: 275.00000001
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 1411396.576016399
Trying x_HFC: 0.6963230922525787
Trying T: 275.0
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 1411386.2145425607
Trying x_HFC: 0.6963231022525788
Trying T: 275.0
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 1411386.2140858825
Trying x_HFC: 0.6963230922525787
Trying T: 275.00000001
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 1411386.2124967757
Trying x_HFC: 0.6959260523660958
Trying T: 275.0
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 1411396.6242656785
Trying x_HFC: 0.6959260623660959
Trying T: 275.0
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 1411396.6244463036
Trying x_HFC: 0.6959260523660958
Trying T: 275.00000001
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 1411396.622763144
Trying x_HFC: 0.6961880986911746
Trying 

In [7]:
results

      fun: -1411396.3242058037
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([  9930.22690194, 225186.33230636])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 75
      nit: 8
     njev: 25
   status: 0
  success: True
        x: array([  0.69596872, 275.        ])

In [10]:
var_guess = (0.1,280)
bnds = ((0.0755,0.8),(272.5,360))
results = scipy.optimize.minimize(error,var_guess,bounds=bnds,tol=1E-4)

Trying x_HFC: 0.1
Trying T: 280.0
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 613.128450262816
Trying x_HFC: 0.10000001
Trying T: 280.0
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 613.1286900577907
Trying x_HFC: 0.1
Trying T: 280.00000001
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 613.1284473110235
Trying x_HFC: 0.8
Trying T: 272.5
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 1275176.3164517323
Trying x_HFC: 0.79999999
Trying T: 272.5
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 1275176.4928401718
Trying x_HFC: 0.8
Trying T: 272.50000001
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 1275176.313500707
Trying x_HFC: 0.10130993248248821
Trying T: 279.98596500911617
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 648.6641202500854
Trying x_HFC: 0.1013099424824882
Trying T: 279.98596500911617
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 648.6643709100276
Trying x_HFC: 0.1013099324

Feasible/Optimal Solution
Feasible/Optimal Solution
error: 1949223.3977930367
Trying x_HFC: 0.710619765493581
Trying T: 272.50000001
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 1949223.3790630847
Trying x_HFC: 0.7152948300936157
Trying T: 272.5
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 1951916.2344420743
Trying x_HFC: 0.7152948400936158
Trying T: 272.5
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 1951915.9141592658
Trying x_HFC: 0.7152948300936157
Trying T: 272.50000001
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 1951916.133876392
Trying x_HFC: 0.7136811924519655
Trying T: 272.5
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 1951333.3833014497
Trying x_HFC: 0.7136812024519655
Trying T: 272.5
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 1951331.1257150746
Trying x_HFC: 0.7136811924519655
Trying T: 272.50000001
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 1951330.2414116922
Trying x_HFC: 

Feasible/Optimal Solution
Feasible/Optimal Solution
error: 1951916.1166041433
Trying x_HFC: 0.7152948300936157
Trying T: 272.5
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 1951916.2344420743
Trying x_HFC: 0.7152948400936158
Trying T: 272.5
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 1951915.9141592658
Trying x_HFC: 0.7152948300936157
Trying T: 272.50000001
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 1951916.133876392
Trying x_HFC: 0.7152948300936156
Trying T: 272.5
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 1951916.1666893703
Trying x_HFC: 0.7152948400936157
Trying T: 272.5
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 1951916.162532164
Trying x_HFC: 0.7152948300936156
Trying T: 272.50000001
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 1951916.1166041433
Trying x_HFC: 0.0755
Trying T: 272.5
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 1485.3982428238817
Trying x_HFC: 0.07550000999999999

In [11]:
results 

      fun: -1951916.2344420743
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([32028280.68956547, 10056560.26211617])
  message: 'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 189
      nit: 7
     njev: 63
   status: 2
  success: False
        x: array([  0.71529483, 272.5       ])

In [15]:
var_guess = (0.6,280)
bnds = ((0.15,0.8),(265,360))
results = scipy.optimize.minimize(error,var_guess,bounds=bnds,tol=1E-4)

Trying x_HFC: 0.6
Trying T: 280.0
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 585894.0555505533
Trying x_HFC: 0.60000001
Trying T: 280.0
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 585894.0736538594
Trying x_HFC: 0.6
Trying T: 280.00000001
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 585894.0546589325
Trying x_HFC: 0.8
Trying T: 265.0
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 3909478.2620733986
Trying x_HFC: 0.79999999
Trying T: 265.0
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 3909478.377600572
Trying x_HFC: 0.8
Trying T: 265.00000001
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 3909478.2583115497
Trying x_HFC: 0.15
Trying T: 265.0
Feasible/Optimal Solution
Feasible/Optimal Solution
error: 20775.75146432726
Trying x_HFC: 0.15000001
Trying T: 265.0
Initialization Error
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a l

In [16]:
results

      fun: -3909478.2620733986
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([11552717.29237371,   376184.59162583])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 12
      nit: 2
     njev: 4
   status: 0
  success: True
        x: array([  0.8, 265. ])